# Star-galaxy Classification Using Deep Convolutonal Neural Networks

This notebook demonstrates the Convolutional Neural Network used in the paper. Note that this notebook is for demonstration and testing purposes only. In the paper, we used a much larger training set of 50,000 images and a test set of 15,000 images.

If you are using a GPU, uncomment the following code cell to write the Theano configuration file, `~/.theanorc`.

In [1]:
'''
%%writefile ~/.theanorc
[global]
device=gpu
floatX=float32

[blas]
ldflags=-lopenblas

[cuda]
root=/opt/apps/cuda/7.0

[nvcc]
fastmath=True
'''

'\n%%writefile ~/.theanorc\n[global]\ndevice=gpu\nfloatX=float32\n\n[blas]\nldflags=-lopenblas\n\n[cuda]\nroot=/opt/apps/cuda/7.0\n\n[nvcc]\nfastmath=True\n'

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import sys
import time
import cPickle as pickle

import numpy as np

import theano
import theano.tensor as T

import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from lasagne.objectives import categorical_crossentropy
from lasagne.nonlinearities import leaky_rectify
from lasagne.init import Orthogonal, Constant
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import BatchIterator
from lasagne.nonlinearities import softmax
from sklearn.utils import shuffle

/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


This example notebook uses only 100 training images. Run [fetch_sdss.py](https://github.com/EdwardJKim/dl4astro/blob/master/scripts/fetch_sdss.py) to create a larger training set. See also [fetch_sdss.ipynb](https://github.com/EdwardJKim/dl4astro/blob/master/notebooks/fetch_sdss.ipynb).

The training set has already been shuffled, so we simply use the first 80 images to train and the remaining 20 images for validation.

In [3]:
X = np.load("../data/sdss_training_images.npy")
print("X.shape = {}, X.min = {}, X.max = {}".format(X.shape, X.min(), X.max()))
y = np.load("../data/sdss_training_labels.npy")
print("y.shape = {}, y.min = {}, y.max = {}".format(y.shape, y.min(), y.max()))

X.shape = (100, 5, 48, 48), X.min = 16.8095626831, X.max = 25.861333847
y.shape = (100,), y.min = -1.0, y.max = 1.0


In [4]:
def renormalize(array):
    return (array - array.min()) / (array.max() - array.min())

for i in range(5):
    X[:, i, :, :] = renormalize(X[:, i, :, :])

y = renormalize(y).astype(np.int32)
print("X.shape = {}, X.min = {}, X.max = {}".format(X.shape, X.min(), X.max()))
print("y.shape = {}, y.min = {}, y.max = {}".format(y.shape, y.min(), y.max()))

X.shape = (100, 5, 48, 48), X.min = 0.0, X.max = 1.0
y.shape = (100,), y.min = 0, y.max = 1


In [5]:
def compute_PCA(array):

    nimages0, nchannels0, height0, width0 = array.shape
    rolled = np.transpose(array, (0, 2, 3, 1))
    # transpose from N x channels x height x width  to  N x height x width x channels
    nimages1, height1, width1, nchannels1 = rolled.shape
    # check shapes
    assert nimages0 == nimages1
    assert nchannels0 == nchannels1
    assert height0 == height1
    assert width0 == width1
    # flatten
    reshaped = rolled.reshape(nimages1 * height1 * width1, nchannels1)

    from sklearn.decomposition import PCA

    pca = PCA()
    pca.fit(reshaped)

    cov = pca.get_covariance()

    eigenvalues, eigenvectors = np.linalg.eig(cov)

    return eigenvalues, eigenvectors

In [6]:
class AugmentedBatchIterator(BatchIterator):

    def __init__(self, batch_size, crop_size=8, testing=False):
        super(AugmentedBatchIterator, self).__init__(batch_size)
        self.crop_size = crop_size
        self.testing = testing

    def transform(self, Xb, yb):

        Xb, yb = super(AugmentedBatchIterator, self).transform(Xb, yb)
        batch_size, nchannels, width, height = Xb.shape

        if self.testing:
            if self.crop_size % 2 == 0:
                right = left = self.crop_size // 2
            else:
                right = self.crop_size // 2
                left = self.crop_size // 2 + 1
            X_new = Xb[:, :, right: -left, right: -left]
            return X_new, yb

        eigenvalues, eigenvectors = compute_PCA(Xb)

        # Flip half of the images horizontally at random
        indices = np.random.choice(batch_size, batch_size // 2, replace=False)
        Xb[indices] = Xb[indices, :, :, ::-1]

        # Crop images
        X_new = np.zeros(
            (batch_size, nchannels, width - self.crop_size, height - self.crop_size),
            dtype=np.float32
        )

        for i in range(batch_size):
            # Choose x, y pixel posiitions at random
            px, py = np.random.choice(self.crop_size, size=2)

            sx = slice(px, px + width - self.crop_size)
            sy = slice(py, py + height - self.crop_size)

            # Rotate 0, 90, 180, or 270 degrees at random
            nrotate = np.random.choice(4)

            # add random color perturbation
            alpha = np.random.normal(loc=0.0, scale=0.5, size=5)
            noise = np.dot(eigenvectors, np.transpose(alpha * eigenvalues))

            for j in range(nchannels):
                X_new[i, j] = np.rot90(Xb[i, j, sx, sy] + noise[j], k=nrotate)

        return X_new, yb

In [7]:
class SaveParams(object):

    def __init__(self, name):
        self.name = name

    def __call__(self, nn, train_history):
        if train_history[-1]["valid_loss_best"]:
            nn.save_params_to("{}.params".format(self.name))
            with open("{}.history".format(self.name), "w") as f:
                pickle.dump(train_history, f)

In [8]:
class UpdateLearningRate(object):

    def __init__(self, start=0.001, stop=0.0001):
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = np.float32(self.ls[epoch - 1])
        getattr(nn, "update_learning_rate").set_value(new_value)

In [9]:
class TrainSplit(object):

    def __init__(self, eval_size):
        self.eval_size = eval_size

    def __call__(self, X, y, net):
        if self.eval_size:
            X_train, y_train = X[:-self.eval_size], y[:-self.eval_size]
            X_valid, y_valid = X[-self.eval_size:], y[-self.eval_size:]
        else:
            X_train, y_train = X, y
            X_valid, y_valid = _sldict(X, slice(len(y), None)), y[len(y):]

        return X_train, X_valid, y_train, y_valid

In [10]:
net = NeuralNet(
    layers=[
        ('input', layers.InputLayer),

        ('conv11', layers.Conv2DLayer),
        ('conv12', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),

        ('conv21', layers.Conv2DLayer),
        ('conv22', layers.Conv2DLayer),
        ('conv23', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),

        ('conv31', layers.Conv2DLayer),
        ('conv32', layers.Conv2DLayer),
        ('conv33', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),

        ('dropout4', layers.DropoutLayer),
        ('hidden4', layers.DenseLayer),

        ('dropout5', layers.DropoutLayer),
        ('hidden5', layers.DenseLayer),

        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 5, 44, 44),

    conv11_num_filters=32, conv11_filter_size=(5, 5),
    conv11_nonlinearity=leaky_rectify,
    conv11_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv11_b=Constant(0.1),

    conv12_num_filters=32, conv12_filter_size=(3, 3), conv12_pad=1,
    conv12_nonlinearity=leaky_rectify,
    conv12_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv12_b=Constant(0.1),

    pool1_pool_size=(2, 2),

    conv21_num_filters=64, conv21_filter_size=(3, 3), conv21_pad=1,
    conv21_nonlinearity=leaky_rectify,
    conv21_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv21_b=Constant(0.1),

    conv22_num_filters=64, conv22_filter_size=(3, 3), conv22_pad=1,
    conv22_nonlinearity=leaky_rectify,
    conv22_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv22_b=Constant(0.1),
    
    conv23_num_filters=64, conv23_filter_size=(3, 3), conv23_pad=1,
    conv23_nonlinearity=leaky_rectify,
    conv23_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv23_b=Constant(0.1),

    pool2_pool_size=(2, 2),

    conv31_num_filters=128, conv31_filter_size=(3, 3), conv31_pad=1,
    conv31_nonlinearity=leaky_rectify,
    conv31_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv31_b=Constant(0.1),

    conv32_num_filters=128, conv32_filter_size=(3, 3), conv32_pad=1,
    conv32_nonlinearity=leaky_rectify,
    conv32_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv32_b=Constant(0.1),

    conv33_num_filters=128, conv33_filter_size=(3, 3), conv33_pad=1,
    conv33_nonlinearity=leaky_rectify,
    conv33_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), conv33_b=Constant(0.1),

    pool3_pool_size=(2, 2),

    hidden4_num_units=2048,
    hidden4_nonlinearity=leaky_rectify,
    hidden4_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), hidden4_b=Constant(0.01),
    dropout4_p=0.5,

    hidden5_num_units=2048,
    hidden5_nonlinearity=leaky_rectify,
    hidden5_W=Orthogonal(np.sqrt(2 / (1 + 0.01**2))), hidden5_b=Constant(0.01),
    dropout5_p=0.5,

    output_num_units=2,
    output_nonlinearity=softmax,

    update_learning_rate=theano.shared(np.float32(0.003)),
    update_momentum=0.9,

    objective_loss_function=categorical_crossentropy,
    regression=False,
    max_epochs=100,
    batch_iterator_train=AugmentedBatchIterator(batch_size=128, crop_size=4),
    batch_iterator_test=AugmentedBatchIterator(batch_size=128, crop_size=4, testing=True),

    on_epoch_finished=[
        UpdateLearningRate(start=0.003, stop=0.0001),
        SaveParams("net")
    ],

    verbose=2,
    train_split=TrainSplit(eval_size=20)
    )

In [11]:
net.fit(X, y)

# Neural Network with 11230754 learnable parameters

## Layer information

name      size         total    cap.Y    cap.X    cov.Y    cov.X
--------  ---------  -------  -------  -------  -------  -------
input     5x44x44       9680   100.00   100.00   100.00   100.00
conv11    32x40x40     51200   100.00   100.00    11.36    11.36
conv12    32x40x40     51200    42.86    42.86    15.91    15.91
pool1     32x20x20     12800    42.86    42.86    15.91    15.91
conv21    64x20x20     25600    54.55    54.55    25.00    25.00
conv22    64x20x20     25600    40.00    40.00    34.09    34.09
conv23    64x20x20     25600    31.58    31.58    43.18    43.18
pool2     64x10x10      6400    31.58    31.58    43.18    43.18
conv31    128x10x10    12800    44.44    44.44    61.36    61.36
conv32    128x10x10    12800    34.29    34.29    79.55    79.55
conv33    128x10x10    12800    27.91    27.91    97.73    97.73
pool3     128x5x5       3200    27.91    27.91    97.73    97.73
dropout4  128x5

/usr/local/lib/python2.7/dist-packages/lasagne/layers/conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


NeuralNet(X_tensor_type=None,
     batch_iterator_test=<__main__.AugmentedBatchIterator object at 0x7fdf2435ca50>,
     batch_iterator_train=<__main__.AugmentedBatchIterator object at 0x7fdf24404710>,
     check_input=True,
     conv11_W=<lasagne.init.Orthogonal object at 0x7fdf24404210>,
     conv11_b=<lasagne.init.Constant object at 0x7fdf24404250>,
     conv11_filter_size=(5, 5),
     conv11_nonlinearity=<lasagne.nonlinearities.LeakyRectify object at 0x7fdf261a6410>,
     conv11_num_filters=32,
     conv12_W=<lasagne.init.Orthogonal object at 0x7fdf24404290>,
     conv12_b=<lasagne.init.Constant object at 0x7fdf244042d0>,
     conv12_filter_size=(3, 3),
     conv12_nonlinearity=<lasagne.nonlinearities.LeakyRectify object at 0x7fdf261a6410>,
     conv12_num_filters=32, conv12_pad=1,
     conv21_W=<lasagne.init.Orthogonal object at 0x7fdf24404310>,
     conv21_b=<lasagne.init.Constant object at 0x7fdf24404350>,
     conv21_filter_size=(3, 3),
     conv21_nonlinearity=<lasagne.nonlinea

In [12]:
best_valid_loss = min([row['valid_loss'] for row in net.train_history_])
print("Best valid loss: {}".format(best_valid_loss))

Best valid loss: 0.611180422227


In [13]:
X_valid = X[-20:]
y_valid = y[-20:]

for i in range(5):
    X_valid[:, i, :, :] = renormalize(X_valid[:, i, :, :])

y_valid = renormalize(y_valid).astype(np.int32)

y_pred_valid = np.zeros((len(y_valid), 64))

In [15]:
class AugmentedBatchIterator(BatchIterator):

    def __init__(self, batch_size, crop_size=8, validation=False, testing=False, startx=None, starty=None, rotate=None):
        super(AugmentedBatchIterator, self).__init__(batch_size)
        self.crop_size = crop_size
        self.validation = validation
        self.testing = testing
        self.startx, self.starty = startx, starty
        self.rotate = rotate

    def transform(self, Xb, yb):

        Xb, yb = super(AugmentedBatchIterator, self).transform(Xb, yb)
        batch_size, nchannels, width, height = Xb.shape

        if self.validation:
            if self.crop_size % 2 == 0:
                right = left = self.crop_size // 2
            else:
                right = self.crop_size // 2
                left = self.crop_size // 2 + 1
            X_new = Xb[:, :, right: -left, right: -left]
            return X_new, yb

        if not self.testing:
            eigenvalues, eigenvectors = compute_PCA(Xb)

        # Flip half of the images horizontally at random
        indices = np.random.choice(batch_size, batch_size // 2, replace=False)
        Xb[indices] = Xb[indices, :, :, ::-1]

        # Crop images
        X_new = np.zeros(
            (batch_size, nchannels, width - self.crop_size, height - self.crop_size),
            dtype=np.float32
        )

        for i in range(batch_size):
            if self.testing:
                px, py = self.startx, self.starty
            else:
                # Choose x, y pixel posiitions at random
                px, py = np.random.choice(self.crop_size, size=2)

            sx = slice(px, px + width - self.crop_size)
            sy = slice(py, py + height - self.crop_size)

            # Rotate 0, 90, 180, or 270 degrees at random
            if self.testing:
                nrotate = self.rotate
                noise = np.zeros(nchannels)
            else:
                nrotate = np.random.choice(4)
                # add random color perturbation
                alpha = np.random.normal(loc=0.0, scale=0.5, size=5)
                noise = np.dot(eigenvectors, np.transpose(alpha * eigenvalues))

            for j in range(nchannels):
                X_new[i, j] = np.rot90(Xb[i, j, sx, sy] + noise[j], k=nrotate)

        return X_new, yb

In [16]:
count = 0

print("Starting model combination...")

for startx in range(4):
    for starty in range(4):
        for rotate in range(4):

            net.batch_iterator_test=AugmentedBatchIterator(
                batch_size=128,
                crop_size=4,
                testing=True,
                startx=startx,
                starty=starty,
                rotate=rotate
            )
            y_pred_valid[:, count] = net.predict_proba(X_valid)[:, 1]

            count += 1

            print("Iteration: {} / 64".format(count))

Starting model combination...
Iteration: 1 / 64
Iteration: 2 / 64
Iteration: 3 / 64
Iteration: 4 / 64
Iteration: 5 / 64
Iteration: 6 / 64
Iteration: 7 / 64
Iteration: 8 / 64
Iteration: 9 / 64
Iteration: 10 / 64
Iteration: 11 / 64
Iteration: 12 / 64
Iteration: 13 / 64
Iteration: 14 / 64
Iteration: 15 / 64
Iteration: 16 / 64
Iteration: 17 / 64
Iteration: 18 / 64
Iteration: 19 / 64
Iteration: 20 / 64
Iteration: 21 / 64
Iteration: 22 / 64
Iteration: 23 / 64
Iteration: 24 / 64
Iteration: 25 / 64
Iteration: 26 / 64
Iteration: 27 / 64
Iteration: 28 / 64
Iteration: 29 / 64
Iteration: 30 / 64
Iteration: 31 / 64
Iteration: 32 / 64
Iteration: 33 / 64
Iteration: 34 / 64
Iteration: 35 / 64
Iteration: 36 / 64
Iteration: 37 / 64
Iteration: 38 / 64
Iteration: 39 / 64
Iteration: 40 / 64
Iteration: 41 / 64
Iteration: 42 / 64
Iteration: 43 / 64
Iteration: 44 / 64
Iteration: 45 / 64
Iteration: 46 / 64
Iteration: 47 / 64
Iteration: 48 / 64
Iteration: 49 / 64
Iteration: 50 / 64
Iteration: 51 / 64
Iteration:

In [19]:
from utils.bmc import BMC
bmc = BMC()
bmc.fit(y_pred_valid, y_valid)

In [20]:
X_test = np.load("../data/sdss_test_images.npy")
y_test = np.load("../data/sdss_test_labels.npy")

for i in range(5):
    X_test[:, i, :, :] = renormalize(X_test[:, i, :, :])

y_test = renormalize(y_test).astype(np.int32)

print("X_test.shape = {}, X_test.min = {}, X_test.max = {}".format(
    X_test.shape, X_test.min(), X_test.max()
    ))
print("y_test.shape = {}, y_test.min = {}, y_test.max = {}".format(
    y_test.shape, y_test.min(), y_test.max()
    ))

y_pred_test = np.zeros((len(y_test), 64))

X_test.shape = (20, 5, 48, 48), X_test.min = 0.0, X_test.max = 1.0
y_test.shape = (20,), y_test.min = 0, y_test.max = 1


In [21]:
count = 0

for startx in range(4):
    for starty in range(4):
        for rotate in range(4):

            net.batch_iterator_test=AugmentedBatchIterator(
                batch_size=128,
                crop_size=4,
                testing=True,
                startx=startx,
                starty=starty,
                rotate=rotate
            )
            y_pred_test[:, count] = net.predict_proba(X_test)[:, 1]

            count += 1

            print("Iteration: {} / 64".format(count))

Iteration: 1 / 64
Iteration: 2 / 64
Iteration: 3 / 64
Iteration: 4 / 64
Iteration: 5 / 64
Iteration: 6 / 64
Iteration: 7 / 64
Iteration: 8 / 64
Iteration: 9 / 64
Iteration: 10 / 64
Iteration: 11 / 64
Iteration: 12 / 64
Iteration: 13 / 64
Iteration: 14 / 64
Iteration: 15 / 64
Iteration: 16 / 64
Iteration: 17 / 64
Iteration: 18 / 64
Iteration: 19 / 64
Iteration: 20 / 64
Iteration: 21 / 64
Iteration: 22 / 64
Iteration: 23 / 64
Iteration: 24 / 64
Iteration: 25 / 64
Iteration: 26 / 64
Iteration: 27 / 64
Iteration: 28 / 64
Iteration: 29 / 64
Iteration: 30 / 64
Iteration: 31 / 64
Iteration: 32 / 64
Iteration: 33 / 64
Iteration: 34 / 64
Iteration: 35 / 64
Iteration: 36 / 64
Iteration: 37 / 64
Iteration: 38 / 64
Iteration: 39 / 64
Iteration: 40 / 64
Iteration: 41 / 64
Iteration: 42 / 64
Iteration: 43 / 64
Iteration: 44 / 64
Iteration: 45 / 64
Iteration: 46 / 64
Iteration: 47 / 64
Iteration: 48 / 64
Iteration: 49 / 64
Iteration: 50 / 64
Iteration: 51 / 64
Iteration: 52 / 64
Iteration: 53 / 64
It

In [22]:
y_pred = bmc.predict_proba(y_pred_test)
np.save("sdss_convnet_pred.npy", y_pred)

In [25]:
from sklearn.metrics import roc_auc_score
print('Area under the ROC curve: {}'.format(roc_auc_score(y_test, y_pred)))

Area under the ROC curve: 0.88
